In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
import random
import copy
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import torch
import torch.optim as optim
import torchvision
from torchvision.transforms import Resize
import torchvision.transforms as transforms
import torchvision.models as models
import torch.quantization
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import pandas as pd
import time


# use GPU if available
if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available and being used.")
else:
        device = torch.device("cpu")
        print("GPU is not available, using CPU instead.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

GPU is available and being used.
Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Hyperparameters:
learning_rate = 0.001
momentum = 0.9
weight_decay = 1e-3

num_epochs = 15
T_max = num_epochs
eta_min = 1e-5


In [ ]:
# Define transformations for CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download the CIFAR-100 training dataset
download_train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
download_test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

batch_size = 64
# Create DataLoader for training and validation datasets
train_loader = DataLoader(download_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(download_test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:08<00:00, 20.9MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
pretrained_model = models.resnet18(pretrained=True)
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 100)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Functions

In [ ]:
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    model.to(device)

    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)
    return model, optimizer, epoch

def save_checkpoint(model, optimizer, epoch, path):
    # Create the directory if it doesn't exist
    import os
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)

def evaluate(model, data_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation during evaluation
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)  # Get predictions
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class labels

            total += labels.size(0)  # Update total number of samples
            correct += (predicted == labels).sum().item()  # Update number of correct predictions

    accuracy = 100 * correct / total  # Calculate accuracy
    return accuracy

### Load from last check point

In [ ]:
# Example usage before resuming training
checkpoint_path = '/content/drive/My Drive/Colab Notebooks/checkpoints/pretrain_resnet18.pth'
pretrained_model = ResNet18(num_classes=100)

optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=learning_rate, weight_decay=weight_decay)

pretrained_model, optimizer, start_epoch = load_checkpoint(pretrained_model, optimizer, checkpoint_path)
pretrained_model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Create model, schedueler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)


# Verify the model
print("start epoch: ", start_epoch)
for name, param in pretrained_model.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape, param.data.sum())

<ipython-input-12-2f6ac4e18b27>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=device)


start epoch:  14
conv1.weight torch.Size([64, 3, 7, 7]) tensor(-0.0831, device='cuda:0')
bn1.weight torch.Size([64]) tensor(15.1651, device='cuda:0')
bn1.bias torch.Size([64]) tensor(10.5746, device='cuda:0')
layer1.0.conv1.weight torch.Size([64, 64, 3, 3]) tensor(-252.3925, device='cuda:0')
layer1.0.bn1.weight torch.Size([64]) tensor(20.1336, device='cuda:0')
layer1.0.bn1.bias torch.Size([64]) tensor(-1.2524, device='cuda:0')
layer1.0.conv2.weight torch.Size([64, 64, 3, 3]) tensor(-79.8547, device='cuda:0')
layer1.0.bn2.weight torch.Size([64]) tensor(18.7809, device='cuda:0')
layer1.0.bn2.bias torch.Size([64]) tensor(-1.7764, device='cuda:0')
layer1.1.conv1.weight torch.Size([64, 64, 3, 3]) tensor(-170.1751, device='cuda:0')
layer1.1.bn1.weight torch.Size([64]) tensor(19.5954, device='cuda:0')
layer1.1.bn1.bias torch.Size([64]) tensor(-5.7394, device='cuda:0')
layer1.1.conv2.weight torch.Size([64, 64, 3, 3]) tensor(-129.5225, device='cuda:0')
layer1.1.bn2.weight torch.Size([64]) tenso

### Train

In [ ]:
# Initializing parameters with zeroes
total_train = torch.zeros(num_epochs)
correct_train = torch.zeros(num_epochs)
avg_loss_train = torch.zeros(num_epochs)
accuracy_train = torch.zeros(num_epochs)

# TRAINING LOOP
print("START TRAINING........")
train_losses = [] # store training loss for each batch
train_accuracies = [] # store training accuracy for each batch
test_accuracies = [] #store test accuracy after each epoch

for epoch in range(start_epoch, num_epochs):
  pretrained_model.train() # Set the model to training mode
  batch_losses = []
  batch_accuracies = []

  for input, target in train_loader:
      input, target = input.to(device), target.to(device)

      # forward
      output = pretrained_model(input)
      loss = criterion(output, target)

      # backward
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # *** Add gradient clipping here ***
      torch.nn.utils.clip_grad_norm_(pretrained_model.parameters(), max_norm=1)


      # save data
      batch_losses.append(loss.item())
      _, predicted = output.max(1)
      total = target.size(0)
      correct = predicted.eq(target).sum().item()
      batch_accuracies.append(100. * correct / total)

  train_losses.append(batch_losses) # append the batch losses for this epoch to the main list
  train_accuracies.append(batch_accuracies) # append the batch accuracies for this epoch to the main list
  avg_loss_train[epoch] = np.mean(batch_losses) # calculate and store average loss for the epoch
  accuracy_train[epoch] = np.mean(batch_accuracies) # calculate and store average accuracy for the epoch

  #Validation after each epoch
  test_accuracy = evaluate(pretrained_model, test_loader, device)
  test_accuracies.append(test_accuracy)

  checkpoint_path = '/content/drive/My Drive/Colab Notebooks/checkpoints/pretrain_resnet18.pth'
  if (epoch + 1) % 2 == 1:
        save_checkpoint(pretrained_model, optimizer, epoch, checkpoint_path)
  print(f"Epoch [{epoch+1}/{num_epochs}] - "
        f"Train Loss: {avg_loss_train[epoch]:.4f} - "
        f"Train Accuracy: {accuracy_train[epoch]:.2f}% - "
        f"Validation Accuracy: {test_accuracy:.2f}% "
        )

  scheduler.step()


START TRAINING........
Epoch [5/15] - Train Loss: 0.4490 - Train Accuracy: 86.17% - Validation Accuracy: 75.79% 
Epoch [6/15] - Train Loss: 0.3755 - Train Accuracy: 88.55% - Validation Accuracy: 76.11% 
Epoch [7/15] - Train Loss: 0.3064 - Train Accuracy: 90.49% - Validation Accuracy: 76.17% 
Epoch [8/15] - Train Loss: 0.2510 - Train Accuracy: 92.37% - Validation Accuracy: 77.21% 
Epoch [9/15] - Train Loss: 0.1936 - Train Accuracy: 94.18% - Validation Accuracy: 76.82% 
Epoch [10/15] - Train Loss: 0.1532 - Train Accuracy: 95.52% - Validation Accuracy: 77.17% 
Epoch [11/15] - Train Loss: 0.1198 - Train Accuracy: 96.73% - Validation Accuracy: 77.48% 
Epoch [12/15] - Train Loss: 0.0929 - Train Accuracy: 97.57% - Validation Accuracy: 77.91% 
Epoch [13/15] - Train Loss: 0.0733 - Train Accuracy: 98.17% - Validation Accuracy: 78.18% 
Epoch [14/15] - Train Loss: 0.0570 - Train Accuracy: 98.65% - Validation Accuracy: 78.32% 
Epoch [15/15] - Train Loss: 0.0508 - Train Accuracy: 98.91% - Validation